# CHAPTER 3 Dictionaries and sets
---
파이썬에서 dict 클래스는 중요하므로 상당히 최적화되어 있다. 파이썬의 고성능 딕셔너리 뒤에는 해시테이블이라는 엔진이 있다. 집합도 해시 테이블을 이용해서 구현하므로, 이 장에서는 집합도 다룬다. 해시 테이블이 작동하는 방식을 알아야 딕셔너리와 집합을 최대로 활용할 수 있다.

이 장에서는 다음과 같은 내용을 설명한다.
* 공통적으로 사용되는 딕셔너리 메서드
* 없는 키에 대한 특별 처리
* 표준 라이브러리에서 제공하는 다양한 딕셔너리 클래스
* set과 frozenset형
* 해시 테이블의 작동 방식
* 해시 테이블의 의미(키 자료형 제한, 예측할 수 없는 순서 등)

## 3.1 일반적인 매핑형
collections.abc 모듈은 dict 및 이와 유사한 자료형의 인터페이스를 정의하기 위해 Mapping 및 MutableMapping 추상 베이스 클래스(ABC)를 제공한다. 추상 베이스 클래스는 매핑이 제공해야 하는 최소한의 인터페이스를 정의하고 문서화하기 위한 것이며, 넓은 의미의 매핑을 지원해야 하는 코드에서 `isinstance()` 테스트를 하기 위한 기준으로 사용된다.

In [1]:
my_dict = {}
import collections
isinstance(my_dict, collections.abc.Mapping)

True

표준 라이브러리에서 제공하는 매핑형은 모두 dict를 이용해서 구현하므로, 키가 ***해시 가능***해야 한다는 제한을 갖고 있다.
>***해시 가능하다(hashable)는 말의 의미는?***
>
수명 주기 동안 결코 변하지 않는 해시값을 갖고 있고(`__hash__()` 메서드가 필요) 다른 객체와 비교할 수 있으면(`__eq__()` 메서드가 필요), 객체를 해시 가능하다고 한다. 동일하다고 판단되는 객체는 반드시 해시값이 동일해야 한다.

In [2]:
tt = (1, 2, (30, 40))
hash(tt)

8027212646858338501

In [4]:
tl = (1, 2, [30, 40])
hash(t1)

TypeError: unhashable type: 'list'

In [6]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

-4118419923444501110

In [8]:
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three':3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

## 3.2 지능형 딕셔너리

## 3.3 공통적인 매핑 메서드
다음 표는 dict와 dict의 변형 중 가장 널리 사용되는 defaultdict와 OrderedDict 클래스가 구현하는 메서드를 보여준다.
![dict-api](./3-1.png)

### 3.3.1 존재하지 않는 기를 setdefault()로 처리하기
fail-fast 철학에 따라, 존재하지 않는 키 k로 d[k]를 접근하면 dict는 오류를 발생시킨다. KeyError를 처리하는 것보다 기본값을 사용하는 것이 더 편리한 경우에는 d.get(k, default)를 사용한다는 것은 파이썬 개발자라면 누구나 알고 있을 것이다. 그렇지만 발견한 값을 갱신할 때, 해당 객체가 가변 객체면 `__getitem__()`이나 `get()` 메서드는 보기 어색하며, 효율성이 좋지 않다. 다음은 `dict.get()`이 좋지 않은 사례를 보여준다.

In [12]:
import sys
import re

WORD_RE = re.compile('\w+')

index = {}
with open("./example-code/03-dict-set/zen.txt", encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            # this is ugly; coded like this to make a point
            occurrences = index.get(word, [])  # <1>
            occurrences.append(location)       # <2>
            index[word] = occurrences          # <3>

# print in alphabetical order
for word in sorted(index, key=str.upper):  # <4>
    print(word, index[word])

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
Complex [(6, 1)]
complex [(5, 23)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m

In [14]:
sorted(index, key=str.upper)

['a',
 'Although',
 'ambiguity',
 'and',
 'are',
 'aren',
 'at',
 'bad',
 'be',
 'beats',
 'Beautiful',
 'better',
 'break',
 'by',
 'cases',
 'Complex',
 'complex',
 'complicated',
 'counts',
 'dense',
 'do',
 'Dutch',
 'easy',
 'enough',
 'Errors',
 'explain',
 'Explicit',
 'explicitly',
 'face',
 'first',
 'Flat',
 'good',
 'great',
 'guess',
 'hard',
 'honking',
 'idea',
 'If',
 'implementation',
 'implicit',
 'In',
 'is',
 'it',
 'let',
 'may',
 'more',
 'Namespaces',
 'nested',
 'never',
 'not',
 'now',
 'Now',
 'obvious',
 'of',
 'often',
 'one',
 'only',
 'pass',
 'Peters',
 'practicality',
 'preferably',
 'purity',
 'Python',
 're',
 'Readability',
 'refuse',
 'right',
 'rules',
 's',
 'should',
 'silenced',
 'silently',
 'Simple',
 'Sparse',
 'Special',
 'special',
 't',
 'temptation',
 'than',
 'that',
 'the',
 'The',
 'There',
 'those',
 'Tim',
 'to',
 'ugly',
 'unless',
 'Unless',
 'way',
 'you',
 'Zen']

In [15]:
import sys
import re

WORD_RE = re.compile('\w+')

index = {}
with open("./example-code/03-dict-set/zen.txt", encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            # this is ugly; coded like this to make a point
            index.setdefault(word, []).append(location)

# print in alphabetical order
for word in sorted(index, key=str.upper):  # <4>
    print(word, index[word])

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
Complex [(6, 1)]
complex [(5, 23)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m